# Code and Documentation
### Part 2 - Gene Signature
____
#### GroupD: Nicole Guobadia, Dhruvi Kaswala, Arashpreet Singh Pandher, Karen Angella Edy Setiawan

Note: make sure to shut down all other kernels

In [1]:
install.packages('glmnet', dependencies=TRUE)
library(glmnet)


  There is a binary version available but the source version is later:
       binary source needs_compilation
glmnet  4.1-1  4.1-6              TRUE

  Binaries will be installed
package 'glmnet' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'glmnet'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\nicol\Anaconda3\Lib\R\library\00LOCK\glmnet\libs\x64\glmnet.dll to C:\Users\nicol\Anaconda3\Lib\R\library\glmnet\libs\x64\glmnet.dll: Permission denied"
Warning message:
"restored 'glmnet'"



The downloaded binary packages are in
	C:\Users\nicol\AppData\Local\Temp\RtmpwDvlQe\downloaded_packages


Warning message:
"package 'glmnet' was built under R version 3.6.3"
Loading required package: Matrix

Loaded glmnet 4.1-1



First, load our training data from the previous notebook (part 1).

In [2]:
load(path.expand("data.RData"))
ls()

[1] "test_dataset"  "train_dataset"

In [3]:
set.seed(1)

For our gene signature, we will first perform t-test for each genes. Then, we adjusted the resulting p values using fdr. We are looking at genes with adjusted p value smaller than 0.05.

In [4]:
# get index for AML and ALL samples
aml.ind <- which(train_dataset$label == 1);
all.ind <- which(train_dataset$label == 0);

pval <- NULL

# do t-test for each gene 
for (j in 1:12708){
    pval[j] <- t.test (train_dataset[c(aml.ind), j], train_dataset[c(all.ind), j])$p.value;
}

In [5]:
# adjust p value using fdr
pval.adj <- p.adjust(pval, method = "fdr")
length(which(pval.adj < 0.05))

[1] 11173

Furthermore, we do feature selection using Mean Absolute Difference (MAD). We are looking for genes with MAD > 1.5. There are 429 genes that has BOTH adjusted p value < 0.05 and MAD > 1.5.

In [6]:
# reference: https://www.geeksforgeeks.org/feature-selection-techniques-in-machine-learning/

# calculate the mean of each gene per aml group
avg.aml = apply(train_dataset[c(aml.ind), ], 2, mean)

# calcuate the mean of each gene per all group
avg.all = apply(train_dataset[c(all.ind), ], 2, mean) 

# mean absolute difference
mad <- abs(avg.aml - avg.all)

In [7]:
# intersecting genes
genes <- intersect(which(mad > 1.5), which(pval.adj < 0.05))
length(genes)

[1] 429

Next, we do gene selection via LASSO penalized regression. Genes with non-zero coefficients at the LASSO penalization level that optimizes cross-validation error are selected.

In [8]:
# lasso genes, penalized maximum likelihood
lasso <- cv.glmnet(as.matrix(train_dataset[1:(ncol(train_dataset) - 1)]), as.matrix(train_dataset['label']), alpha=1)

In [9]:
lassogene <- data.frame(coef.name = dimnames(coef(lasso))[[1]], coef.value = matrix(coef(lasso)))
lassogene <- lassogene[-1,]

In [10]:
length(which(lassogene[,'coef.value'] != 0))

[1] 287

In [11]:
# looking for genes with coef != 0
lassogene <- which(lassogene[,'coef.value'] != 0)

Our final gene signature, are genes that has all three: 
1. adjusted p value < 0.05
2. MAD > 1.5
3. non-zero coefficient at LASSO penalization.

And we have 59 genes.

In [12]:
# gene signature: gene that fulfills pval < 0.05, mad > 1.5, and coef != 0
genesig <- intersect(genes, lassogene)
length(genesig)

[1] 59

In [13]:
# saving gene signature so it can be loaded to another ipynb file
save(genesig, file = "gene.RData")